# This is my note of study SqueezeDet 

## Importing the libraries

In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
from datetime import datetime
import os.path
import sys
import time

import numpy as np
from six.moves import xrange
import tensorflow as tf
import threading

from config import *
from dataset import pascal_voc, kitti
from utils.util import sparse_to_dense, bgr_to_rgb, bbox_transform
from nets import *

## Determine if train_dir is exist, if it is exist will delete it.
## Then make a file for training

In [ ]:
def main(argv=None): 
    if tf.gfile.Exists(FLAGS.train_dir): #
        tf.gfile.DeleteRecursively(FLAGS.train_dir)

    tf.gfile.MakeDirs(FLAGS.train_dir)
    train()

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 tf.gfile.DeleeRecursively & tf.gfile.MakeDirs
</font></td></tr></table>
<br>



<font face="微软雅黑" size=6 color='0000ff'> 1. Let us know "tf.gfile"<br> </font>
<br>
 
<font face="微软雅黑" size=4 color='000000'>
The main roles of the tf.gfile module are:
    <br>
    *:To provide an API that is close to Python's file objects <br>
    *:To provide an implementation based on TensorFlow's C++ filesystem API
<br> 
</font>

<br>

<font face="微软雅黑" size=6 color='0000ff'>2. Let us know "tf.gfile.DeleeRecursively"  <br></font>
<br>

<font face="微软雅黑" size=4 color='000000'>
tf.gfile.DeleteRecursively(dirname)

Defined in tensorflow/python/lib/io/file_io.py.

Deletes everything under dirname recursively.
Args:

    *dirname: string, a path to a directory

Raises:

    *errors.OpError: If the operation fails.
 </font>
<br>   
    
    
<font face="微软雅黑" size=6 color='0000ff'> 3. Let us know "tf.gfile.MakeDirs" </font><br>  

<font face="微软雅黑" size=4 color='#000000'>    
tf.gfile.MakeDirs(dirname)

Defined in tensorflow/python/lib/io/file_io.py.

Creates a directory and all parent/intermediate directories.

It succeeds if dirname already exists and is writable.
Args:

    *dirname: string, name of the directory to be created

Raises:

    *errors.OpError: If the operation fails.

 </font>
<br>  

## Setting 'CUDA_VISIBLE_DEVICES' to use  witch GPU for trianing

In [ ]:
def train():
  """Train SqueezeDet model"""
  assert FLAGS.dataset == 'KITTI', \
      'Currently only support KITTI dataset'

  os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 assert
</font></td></tr></table>
<br>
Python's assert statement helps you find bugs more quickly and with less pain. This note has some suggestions on good ways to use it. 

## For example

In [6]:
dataset = 'dd'
assert dataset == 'KITTI', \
      'Currently only support KITTI dataset'

AssertionError: Currently only support KITTI dataset

## Define operations and tensors use tf.Graph().as_default().

In [ ]:
with tf.Graph().as_default():

<table><tr><td bgcolor=orange > <font face="purisa" size=8 color='0000ff'>
 Extended learning
</font></td></tr></table>
<br>
<table><tr><td bgcolor=white > <font face="sawasdee" size=8 color='0000ff'>
 with
</font></td></tr></table>
<br>
有一些任务，可能事先需要设置，事后做清理工作。对于这种场景，Python的with语句提供了一种非常方便的处理方式。一个很好的例子是文件处理，你需要获取一个文件句柄，从文件中读取数据，然后关闭文件句柄。
如果不用with语句，代码如下：

file = open("/tmp/foo.txt")
data = file.read()
file.close()

这里有两个问题。一是可能忘记关闭文件句柄；二是文件读取数据发生异常，没有进行任何处理。下面是处理异常的加强版本：

file = open("/tmp/foo.txt")
try:
    data = file.read()
finally:
    file.close()

虽然这段代码运行良好，但是太冗长了。这时候就是with一展身手的时候了。除了有更优雅的语法，with还可以很好的处理上下文环境产生的异常。下面是with版本的代码：

with open("/tmp/foo.txt") as file:
    data = file.read()

## Specify the parameters to tune, such as image width, image height, batch size, weight decay...
## Specify the Layer imformation of SqueezeDet model
## kitti_squeezeDet_config() from ./scr/config/kitti_squeezeDet_config.py
## SqueezeDet() from ./scr/nets/SqueezeDet

In [18]:
    ## elif FLAGS.net == 'squeezeDet':
mc = kitti_squeezeDet_config()
mc.IS_TRAINING = True
mc.PRETRAINED_MODEL_PATH = '../squeezeDet-master/data/SqueezeNet/squeezenet_v1.1.pkl'#FLAGS.pretrained_model_path
model = SqueezeDet(mc)

Cannot find fire10/squeeze1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire10/expand1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire10/expand3x3 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/squeeze1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/expand1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/expand3x3 in the pretrained model. Use randomly initialized parameters
Cannot find conv12 in the pretrained model. Use randomly initialized parameters
INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.


INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.



imdb = kitti(FLAGS.image_set, FLAGS.data_path, mc)